In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

In [2]:
main_df = pd.read_csv('spacex_launch_dash.csv')
main_df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [3]:
main_df['Payload Mass (kg)'].describe()

count      56.000000
mean     3696.648214
std      2568.509679
min         0.000000
25%      2121.000000
50%      3412.500000
75%      5042.500000
max      9600.000000
Name: Payload Mass (kg), dtype: float64

In [7]:
app = dash.Dash(__name__)

app.layout = html.Div([
    # Dropdown to select a Launch site
    html.Div([
        html.Label('Select Launch Site:'),
        dcc.Dropdown(
            id='site-dropdown',
            options=[{'label': site, 'value': site} for site in main_df['Launch Site'].unique()],
            value=main_df['Launch Site'].unique()[0]  # Default value
        )
    ]),
    
    # Pie chart for success/failure counts
    html.Div([
        dcc.Graph(id='success-failure-pie-chart')
    ]),
    
    # Range slider for selecting payload
    html.Div([
        html.Label('Select Payload Range (kg):'),
        dcc.RangeSlider(
            id='payload-slider',
            min=main_df['Payload Mass (kg)'].min(),
            max=main_df['Payload Mass (kg)'].max(),
            step=100,
            value=[main_df['Payload Mass (kg)'].min(), main_df['Payload Mass (kg)'].max()],
            marks={int(i): str(i) for i in range(int(main_df['Payload Mass (kg)'].min()), int(main_df['Payload Mass (kg)'].max()), 1000)}
        )
    ]),
    
    # Scatter plot for payload vs class
    html.Div([
        dcc.Graph(id='payload-class-scatter')
    ])
])

# Callback to update the pie chart based on the selected Launch site
@app.callback(
    Output('success-failure-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    filtered_df = main_df[main_df['Launch Site'] == selected_site]
    pie_fig = px.pie(
        filtered_df,
        names='class',
        title=f'Success and Failure Counts for {selected_site}',
        hole=.3
    )
    return pie_fig

# Callback to update the scatter plot based on the selected Launch site and payload range
@app.callback(
    Output('payload-class-scatter', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_plot(selected_site, payload_range):
    filtered_df = main_df[(main_df['Launch Site'] == selected_site) &
                          (main_df['Payload Mass (kg)'] >= payload_range[0]) &
                          (main_df['Payload Mass (kg)'] <= payload_range[1])]
    scatter_fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        title=f'Payload vs. Outcome for {selected_site}',
        labels={'class': 'Outcome (0=Failure, 1=Success)'},
        color='Booster Version Category'
    )
    return scatter_fig

if __name__ == '__main__':
    app.run_server(debug=True)